In [1]:
import sys
import pathlib

sys.path.append(r"C:\Users\amrul\programming\deep_learning\dl_projects\Generative_Deep_Learning_2nd_Edition")

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import datasets, transforms
from tqdm import tqdm

IMAGE_SIZE = 64
BATCH_SIZE = 64
DATASET_REPETITIONS = 5
LOAD_MODEL = False

NOISE_EMBEDDING_SIZE = 32
PLOT_DIFFUSION_STEPS = 20

# optimization
EMA = 0.999
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 50


In [3]:
from notebooks.utils import display

In [4]:
import torch
import torch.nn as nn
from ddm_torch_model import DiffusionModel,get_flower_images_train_dataset,convert_images_torch_to_numpy_for_display,training_loop,Normalizer

In [5]:
train_data = get_flower_images_train_dataset()
print(f"Loaded flower train dataset : {len(train_data)}")

Loaded flower train dataset : 6552


In [6]:
image,label=train_data[0]
print(image.shape,type(image))

torch.Size([3, 64, 64]) <class 'torch.Tensor'>


In [7]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)

In [8]:
from ddm_torch_model import calc_mean_and_std
recalculate_means_stds=False

if recalculate_means_stds:
    total_mean = torch.zeros(3)
    total_var = torch.zeros(3)
    for images,_ in tqdm(train_loader):
        mean,std = calc_mean_and_std(images)
        total_mean += mean
        total_var += std**2

    final_mean =total_mean/len(train_loader)
    final_var = total_var/len(train_loader)
    final_std = torch.sqrt(final_var)
    print(f"final mean : {final_mean}, final std : {final_std}")
else:
    final_mean = torch.tensor([0.4352, 0.3771, 0.2871])
    final_std = torch.tensor([0.2853, 0.2328, 0.2580])

In [9]:
final_std

tensor([0.2853, 0.2328, 0.2580])

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [11]:
normalizer = Normalizer()
normalizer.mean=final_mean
normalizer.std = final_std
model = DiffusionModel(3,normalizer,device)
model = model.to(device)

In [12]:
len(model.unet.skips)

0

In [13]:
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(),lr=LEARNING_RATE)

In [14]:
mae_loss = nn.L1Loss()

In [15]:
ret = training_loop(EPOCHS,optimizer,model,mae_loss,train_loader,device)

Epoch loop:   0%|          | 0/50 [00:00<?, ?it/s]c:\Users\amrul\pyvirtualenvs\dl_311_venv\Lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([64, 3, 64, 64])) that is different to the input size (torch.Size([64, 1, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
c:\Users\amrul\pyvirtualenvs\dl_311_venv\Lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([24, 3, 64, 64])) that is different to the input size (torch.Size([24, 1, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
Epoch loop:   2%|▏         | 1/50 [00:59<48:33, 59.46s/it]

Epoch 0 train loss : 0.3464556536049519


Epoch loop:   4%|▍         | 2/50 [01:22<30:25, 38.04s/it]

Epoch 1 train loss : 0.26158577898173657


Epoch loop:   6%|▌         | 3/50 [01:45<24:19, 31.05s/it]

...

Epoch 8 train loss : 0.1643055009755116


Epoch loop:  20%|██        | 10/50 [04:22<15:17, 22.95s/it]

Epoch 9 train loss : 0.1623445657416455


Epoch loop:  22%|██▏       | 11/50 [04:45<14:53, 22.90s/it]

Epoch 10 train loss : 0.15683979468727574
